# Two Agents Collaborating to Solve LeWord Puzzle Game

This notebook demonstrates a multi-agent AI system designed to collaboratively play and solve the LeWord puzzle game. The setup includes two specialized AI agents:

1. **Vision Agent:** Processes visual representations of the game board to understand the current game state, using rendered images of guesses and their feedback.

2. **Word Agent:** Uses the structured game state information combined with the vision agent’s embeddings to decide the next best guess.

The agents interact through a set of tools including guessing, requesting hints, and rendering the game board image as base64. The game state is modeled using Pydantic data classes for structured and type-safe communication. This architecture showcases:

- Multi-agent cooperation  
- Use of vision models to interpret game state visually  
- Structured data exchange with Pydantic schemas  
- Integration of multiple tool functions for flexible AI reasoning

The notebook runs the game loop where the agents iteratively update their understanding and make guesses until the puzzle is solved or attempts are exhausted.


In [7]:
# Enable automatic reloading of modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
import os

# Add the project root (1 level up from 'notebooks/') to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


In [ ]:
import json
from game.leword_game import LeWordGame
from tools.game_tools import guess, available_tools
from agents.vision_agent import vision_agent_process_board
from agents.word_agent import word_agent_decide_guess
from models.schemas import GameState, GuessResult
from PIL import Image
import base64
from io import BytesIO
from vision.render_board import render_leword_board

# Create game instance
the_word = "subaru"  # Example word
the_hint = "Japanese car brand."  # Example hint
game = LeWordGame(the_word, the_hint, 10)

# Initialize tools
functions = available_tools()

def decode_base64_image(img_b64: str) -> Image.Image:
    img_data = base64.b64decode(img_b64)
    return Image.open(BytesIO(img_data))

def run_turn(game):
    if not game.attempts:
        dummy_board = [[' ']*len(game.target_word)]
        board_img = render_leword_board(dummy_board, len(the_word))        
    else:
        board_img = render_leword_board([list(attempt.feedback) for attempt in game.attempts], len(game.target_word))

    # Step 1: Render board and encode image
    vision_embedding = vision_agent_process_board(board_img)

    game_state = GameState(
        attempts=[
            GuessResult(
                guess=attempt.guess,
                feedback=attempt.feedback,
                correct=(attempt.guess.lower() == game.target_word.lower())
            )
            for attempt in game.attempts
        ],
        attempts_left=game.max_attempts - len(game.attempts),
        word_length=len(game.target_word)
    )    

    # Step 3: Word agent decides next guess based on vision + game state
    next_guess = word_agent_decide_guess(vision_embedding, game_state)

    # Step 4: Call guess tool with next_guess
    tool_response = guess(next_guess, game=game)
    print(f"Guess: {next_guess}, Feedback: {tool_response}")

    # Step 5: Check win condition
    game_over = tool_response.get("correct", False)

    return game_over

# Run game loop
for turn in range(game.max_attempts):
    print(f"Turn {turn+1}")
    game_over = run_turn(game)
    if game_over:
        print("🎉 Correct word guessed! Game over.")
        break
else:
    print("❌ Max attempts reached. Game over.")


Turn 1
Guess: honda, Feedback: {'guess': 'honda', 'feedback': 'Invalid length of characters.', 'score': False, 'correct': False}
Turn 2
Guess: subaru, Feedback: {'guess': 'subaru', 'feedback': 'SUBARU', 'score': 60, 'correct': True}
🎉 Correct word guessed! Game over.
